In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/binary-buzz-cyber-labs-ml-25/train.csv
/kaggle/input/binary-buzz-cyber-labs-ml-25/test.csv


In [2]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import balanced_accuracy_score

train_set = pd.read_csv("/kaggle/input/binary-buzz-cyber-labs-ml-25/train.csv").to_numpy()
test_set2 = pd.read_csv("/kaggle/input/binary-buzz-cyber-labs-ml-25/test.csv").to_numpy()

y_label = train_set[ : ,-1]
train_set = train_set[ : , :-1] 

print("The shape of train_set is :", train_set.shape)
print("The shape of test_set2 is :", test_set2.shape)
print("The shape of y_label is", y_label.shape)

2025-12-19 07:48:36.320202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766130516.553850      17 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766130516.620973      17 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766130517.163677      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766130517.163856      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766130517.163861      17 computation_placer.cc:177] computation placer alr

The shape of train_set is : (2000, 3073)
The shape of test_set2 is : (2000, 3073)
The shape of y_label is (2000,)


In [3]:
X_train,X_test,y_train,y_test = train_test_split(train_set, y_label, test_size = 0.2)


print(f"The shape of X_train = {X_train.shape}, X_test = {X_test.shape}")
print(f"The shape of y_train = {y_train.shape}, y_test = {y_test.shape}")

The shape of X_train = (1600, 3073), X_test = (400, 3073)
The shape of y_train = (1600,), y_test = (400,)


In [4]:

model = Sequential([
    Dense( units = 25, activation = 'relu'),
    Dense( units = 10, activation = 'relu'),
    Dense( units = 1, activation = 'linear')
]
)
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(from_logits = True)
)
model.fit(X_train, y_train, epochs = 50)

y_train_prob = model.predict( X_train )

def compute_epsilon(y, y_pred):
    epsilon = np.logspace(-8, 0, 25)
    accuracy = np.zeros_like(epsilon)
    for i, e in enumerate(epsilon):
        y_pred_binary = np.where( y_pred > e, 1, 0 )
        accuracy[i] = balanced_accuracy_score(y, y_pred_binary)

    return epsilon[ np.argmax(accuracy) ]

e = compute_epsilon( y_train, y_train_prob )
y_train_pred = np.where( y_train_prob > e , 1, 0)

print(f"The Maximum Balnced Accuracy for the trained dataset for threshold value of {e}: { balanced_accuracy_score( y_train, y_train_pred ) : .4f}")
y_test_prob = model.predict( X_test )
y_test_pred = np.where( y_test_prob > e, 1 ,0) 
print(f"The Balnced Accuracy for the test dataset for this threshold: {balanced_accuracy_score( y_test, y_test_pred ) : .4f}")

Y = model.predict( test_set2 )
Y_pred = np.where( Y > e, 1, 0)
test_set = np.c_[ test_set2 , Y_pred ]
print("The first five prediction for the given test set : \n", test_set[0:5])

test_df = pd.read_csv("/kaggle/input/binary-buzz-cyber-labs-ml-25/test.csv")

test_ids = test_df['id'].values

submission = pd.DataFrame({
    'id': test_ids, 
    'target': Y_pred.ravel()
})

submission.to_csv('submission.csv', index=False)

print("Success! 'submission.csv' has been created.")

Epoch 1/50


2025-12-19 07:48:55.093208: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 45.3292
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.7643
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4362
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3168
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3431
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2937
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2905
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2435
Epoch 9/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2707
Epoch 10/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2395
Epoch 11/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2515
Epoch 12/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2473
Epoch 13/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2194
Epoch 14/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1961
Epoch 15/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1807
Epoch 16/50
50/50 ━━━━━━━━━━━